In [1]:
import os.path
import numpy as np
import pandas as pd
from time import time
import dask.dataframe as dd
from sklearn.cluster import KMeans
import bcolz
from pandas.computation import expressions as expr

In [2]:
def bcolz_setting():
	# Global settings for bcolz and pandas (for experimenting, modify some of the lines below)
	bcolz.defaults.cparams['cname'] = 'blosclz'
	bcolz.defaults.cparams['clevel'] = 1
	bcolz.defaults.eval_vm = "numexpr"
	bcolz.blosc_set_nthreads(4)
	bcolz.numexpr.set_num_threads(4)
	expr.set_use_numexpr(True)


In [3]:
bcolz.print_versions()
bcolz_setting()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.1.2
NumPy version:     1.12.1
Blosc version:     1.11.2 ($Date:: 2017-01-27 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib', 'zstd']
Numexpr version:   2.6.2
Dask version:      0.14.3
Python version:    3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
Byte-ordering:     little
Detected cores:    8
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


C:\Users\VIPLAB\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: pandas.computation.expressions.set_use_numexpr is deprecated and will be removed in a future version.
you can toggle usage of numexpr via pandas.get_option('compute.use_numexpr')
  


In [ ]:
# set configure
path = "./"
filename = "mds_mly_minus20160901"
relative_filename = path + filename + ".csv"
# out_filename = "mds_mly_minus20160901"
# after the numeric_st_idx's number will be tranform to int64
numeric_st_idx = 2
n_clusters = 5

In [ ]:
# read revise csv file and print cost time
t0 = time()
df = pd.read_csv(relative_filename, error_bad_lines=False)
print("time for read csv: %.2f" % (time()-t0))

In [ ]:
df.info()

In [ ]:
# Create the dataframe
# http://jcrist.github.io/dask-sklearn-part-3.html
# cols = df.columns[2:]
t0 = time()
# dask_df no columns slice, so u should do it in here, use usecols argument.
dask_df = dd.read_csv(relative_filename,
#                  usecols=cols,
                 blocksize=int(128e6),
                 storage_options=dict(anon=True))
print("time for read csv: %.2f" % (time()-t0))

In [ ]:
dask_df.info()

In [ ]:
t0 = time()
kmeans = KMeans(n_clusters=n_clusters).fit(df[df.columns[2:]])
print("time for k mean csv: %.2f " % (time()-t0))

In [ ]:
dask_df.head()

In [ ]:
t0 = time()
kmeans = KMeans(n_clusters=n_clusters).fit(dask_df)
print("time for k mean csv: %.2f " % (time()-t0))

In [88]:
t0 = time()
bcolz_df = bcolz.ctable.fromdataframe(df, names = list(df.columns[2:]))
print("time for ctable conversion: %.2f" % (time()-t0))

time for ctable conversion: 0.14


In [89]:
bcolz_df["MO_OFFNET_DUR"]

carray((1878076,), int64)
  nbytes := 14.33 MB; cbytes := 4.13 MB; ratio: 3.47
  cparams := cparams(clevel=1, shuffle=1, cname='blosclz', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 16384
[ 53 125 260 ...,  52 357  83]

In [91]:
t0 = time()
kmeans = KMeans(n_clusters=n_clusters).fit(bcolz_df)
print("time for k mean csv: %.2f " % (time()-t0))

time for k mean csv: 133.06 


In [ ]:
del pd